### 2 Bookshelf Manipulation

In [143]:
import robotic as ry
import random
import time
max_trials = 10
max_vel = 100
max_acc = max_vel
selected_color = [0, 0, 1]
orig_color = [0.80, 0.80, 0.80]

In [241]:
C = ry.Config()
print(ry.raiPath(''))
C.addFile(ry.raiPath('book_shelf.g'))

C.view()
qHome = C.getJointState()
gripper_orig_pos = C.getFrame("l_gripper").getPosition()
gripper_orig_quat = C.getFrame("l_gripper").getQuaternion()
# print(C.getFrameNames())

objects = ["book1", "book2", "book3", "book4", "mug", "book-non"]

/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/robotic/rai-robotModels/


In [ ]:
# reset colors
for obj in objects:
    C.getFrame(obj).setColor(orig_color)
C.view()







# Reset config
# komo = ry.KOMO()
# del komo
# C.setJointState(qHome)
# C.view()









37

In [244]:
# select an object randomly
curObj = random.choice(objects)

# curObj = "book2"

print("Selected object: ", curObj)

# color the selected object
C.getFrame(curObj).setColor(selected_color)

# determine orientation of object
obj_z_height = C.getFrame(curObj).getSize()[2]
obj_y_width = C.getFrame(curObj).getSize()[1]
obj_x_length = C.getFrame(curObj).getSize()[0]

if min(obj_z_height, obj_y_width, obj_x_length) == obj_z_height:
  obj_orientation = "horizontal"
elif min(obj_z_height, obj_y_width, obj_x_length) == obj_y_width:
  obj_orientation = "vertical"
elif min(obj_z_height, obj_y_width, obj_x_length) == obj_x_length:
  obj_orientation = "sideways"
  
print("Object orientation: ", obj_orientation)


komo = ry.KOMO()
del komo
C.setJointState(qHome)
C.view()

Selected object:  book-non
Object orientation:  horizontal


0

In [227]:
C.addFrame("visual") .setPosition(C.getFrame(curObj).getPosition()) .setColor([1, 0, 0]) .setShape(ry.ST.marker, [0.5])
# C.addFrame("visual_gripper_orig")  .setShape(ry.ST.marker, [0.]).setPosition(gripper_orig_pos) .setQuaternion(gripper_orig_quat) .setColor([0, 1, 0])
C.addFrame("visual_gripper_final") .setShape(ry.ST.marker, [0.3]) .setPosition(C.getFrame("l_gripper").getPosition()) .setQuaternion(C.getFrame("l_gripper").getQuaternion()) .setColor([0, 0, 0]) 
C.view()

-- WARNING:kin.cpp:addFrame:193(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:193(-1) frame already exists! returning existing without modifications!


37

In [245]:
# komo = ry.KOMO(C, phases=1, slicesPerPhase=30, kOrder=1, enableCollisions=False)

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 10, 5., 0)

# add collision constraints
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e0])
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e0], qHome)

# calculate gripper and cargo offset (-40% in x direction)
offset_x = -(C.getFrame(curObj) .getSize()[0] * 0.3)
# this objective moves the gripper close to the cargo
komo.addObjective(
    times=[],
    feature=ry.FS.positionRel,
    frames=['l_gripper', curObj],
    type=ry.OT.eq,
    scale=[1e1],
    target=[offset_x, 0, 0]
)
# this objective makes the gripper claws face the shelf objects 
komo.addObjective(
    times=[0, 1], 
    feature=ry.FS.scalarProductXZ, 
    frames=[curObj, 'l_gripper'], 
    type=ry.OT.eq, 
    scale=[1e1],
    target=[-1]
)


# these 3 objectives align the gripper claws aligned properly with the book
komo.addObjective(
    times=[0, 1], 
    feature=ry.FS.scalarProductXX, 
    frames=['l_gripper', curObj], 
    type=ry.OT.eq, 
    scale=[1e1],
)

komo.addObjective(
    times=[0, 1],
    feature=ry.FS.scalarProductZZ, 
    frames=['l_gripper', curObj], 
    type=ry.OT.eq, 
    scale=[1e1]
)

komo.addObjective(
    times=[0, 1],
    feature=ry.FS.scalarProductYY, 
    frames=['l_gripper', curObj], 
    type=ry.OT.eq, 
    scale=[1e1]
)

# this objective makes the gripper claws vertical

# komo.addObjective(
#     times=[0, 1],
#     feature=ry.FS.scalarProductXY, 
#     frames=['l_gripper', curObj], 
#     type=ry.OT.eq, 
#     scale=[1e1],
#     target=[0]
# )


if obj_orientation == "vertical":
    komo.addObjective(
        times=[0, 1],
        feature=ry.FS.scalarProductXY, 
        frames=['l_gripper', curObj], 
        type=ry.OT.eq, 
        scale=[1e1],
        target=[1]
    )
elif obj_orientation == "horizontal":
    komo.addObjective(
        times=[0, 1],
        feature=ry.FS.scalarProductXY, 
        frames=['l_gripper', curObj], 
        type=ry.OT.eq, 
        scale=[1e1],
        target=[0]
    )
    komo.addObjective(
        times=[0, 1],
        feature=ry.FS.scalarProductYY, 
        frames=['l_gripper', curObj], 
        type=ry.OT.eq, 
        scale=[1e2],
        target=[1]
    )

    




# C.attach('l_gripper', curObj)


# solve
ret_1 = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
q_gripper_to_obj = komo.getPath()

for t in range(q_gripper_to_obj.shape[0]):
    C.setJointState(q_gripper_to_obj[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.01)

# komo.view_play(False, 2)

# if ret_1.feasible:
#     print("komo is feasible")
# else:
#     print("komo is NOT feasible")